<a href="https://colab.research.google.com/github/PeteJoh/fpl-ai/blob/master/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#------------------------------------------------------
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [98]:
forward_statistics =['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
        'mean assists 3','mean bonus 3', 'mean bps 3','mean creativity 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3', 'mean penalties_missed 3',  'mean threat 3',
       'mean total_points 3','mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

leak_columns = [
    "name",
    "team",
]  # columns that shouldnt be used in training fir fear of data leakage


dropped_columns = [
    "season",
    "opponent",
    "match_result",
   # "position",
    "assists",
    "penalties_missed",
    "bonus",
    "bps",
    "clean_sheets",
    "creativity",
    "goals_conceded",
    "goals_scored",
    "ict_index",
    "influence",
    "own_goals",
    "penalties_saved",
    "red_cards",
    "saves",
    "selected",
    "threat",
    "transfers_balance",
    "transfers_in",
    "transfers_out",
    "yellow_cards",
    "team Goal scored",
    "team Goal conceded"
]  # "value",

midfielder_statistics =['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
        'mean assists 3','mean bonus 3', 'mean bps 3','mean creativity 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3', 'mean penalties_missed 3',  'mean threat 3',
       'mean total_points 3','mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

goalkeeper_statistics = ['value', 'was_home', 'last_season_position', 'percent_value',
       'position rank', 'total_points_ex', 'minutes_ex', 'goals_conceded_ex',
       'bonus_ex', 'bps_ex', 'ict_index_ex', 'clean_sheets_ex',
       'red_cards_ex', 'now_cost_ex', 'GW', 'opponent_last_season_position',
       'mean bonus 3', 'mean bps 3', 'mean clean_sheets 3', 'mean goals_conceded 3',
       'mean ict_index 3',  'mean minutes 3',
       'mean own_goals 3',  'mean penalties_saved 3',
        'mean saves 3',  'mean threat 3',
       'mean total_points 3',
       'mean value 3', 'mean match_result 3', 'std bps 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3']

statistics =['value', 'position','was_home', 'last_season_position', 'percent_value',
       'position rank', 'goals_scored_ex', 'assists_ex', 'total_points_ex',
       'minutes_ex', 'goals_conceded_ex', 'creativity_ex', 'influence_ex',
       'threat_ex', 'bonus_ex', 'bps_ex', 'ict_index_ex', 'clean_sheets_ex',
       'yellow_cards_ex','now_cost_ex', 'GW', 'opponent_last_season_position', 'mean assists 3',
       'mean bonus 3', 'mean bps 3', 'mean clean_sheets 3',
       'mean creativity 3', 'mean goals_conceded 3', 'mean goals_scored 3',
       'mean ict_index 3', 'mean influence 3', 'mean minutes 3',
       'mean own_goals 3',
       'mean red_cards 3',  'mean threat 3','mean total_points 3',
       'mean value 3', 'mean match_result 3', 'std bps 3', 'std creativity 3',
       'std ict_index 3', 'std influence 3', 'std minutes 3',
       'std threat 3', 'std total_points 3', 'std value 3','mean saves 3','mean assists all',
       'mean bonus all', 'mean bps all', 'mean clean_sheets all',
       'mean creativity all', 'mean goals_conceded all', 'mean goals_scored all',
       'mean ict_index all', 'mean influence all', 'mean minutes all',
       'mean own_goals all',
       'mean red_cards all',  'mean threat all','mean total_points all',
       'mean value all', 'mean match_result all',
       'mean team Goal scored 3','mean team Goal scored all','mean team Goal conceded 3','mean team Goal conceded all',"ratio_goal_scored all","ratio_goal_scored 3",
       'opp mean team Goal scored 3','opp mean team Goal conceded 3','opp mean team Goal scored all','opp mean team Goal conceded all',"opp mean match_result all"]


date_cols=["day_of week","month","hour","week"]

In [99]:
def convert_minutes(val):
    """CONVERTS MINUTES TO A CATEGORICAL OUTPUT"""
    if val > 10:
        return 1
    else:
        return 0

In [70]:
pip install catboost

In [71]:
from hyperopt import tpe,hp,fmin,STATUS_OK,Trials
from hyperopt.pyll.base import scope

In [100]:
import pandas as pd
import numpy as np
import warnings
import os


warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import (
    RandomForestClassifier,
    RandomForestRegressor,
    GradientBoostingRegressor,
)
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    confusion_matrix,
    accuracy_score,
    f1_score,
)
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,KFold

In [125]:
gameweek=10

In [126]:
# RELOAD THIS DATA FOR EACH MODEL

train = pd.read_csv("/content/gdrive/MyDrive/fplpredict/cleaned_dataset/cleaned_previous_seasons.csv",index_col=0)#/content/drive/MyDrive/Fplpredict /cleaned_previous_seasons.csv", index_col=0)
old_gameweek_cleaned = []
for i in range(1, gameweek):
    old_gameweek_cleaned.append(pd.read_csv(f"/content/gdrive/MyDrive/fplpredict/cleaned_dataset/2024-25/GW{i}.csv"))
old_gameweeks = pd.concat(old_gameweek_cleaned)[train.columns]
train = pd.concat([train, old_gameweeks])

# data for current gameweek we want to predict on
test = pd.read_csv(f"/content/gdrive/MyDrive/fplpredict/cleaned_dataset/2024-25/GW{gameweek}.csv", index_col=0)

In [103]:
#CREATE GAMEWEEK PREDICTIONS FOLDER
path = f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}"

# create new single directory
# check whether directory already exists
if not os.path.exists(path):
    os.mkdir(path)
    print("Folder %s created!" % path)
else:
    print("Folder %s already exists" % path)

# data from current season but previous gameweeks

Folder /content/gdrive/MyDrive/fplpredict/predicted_dataset/GW10 already exists


In [127]:
train["position"].value_counts()

,count
position,
MID,52571
DEF,42209
FWD,15724
GK,13166
GKP,767


In [128]:
train["position"]=train["position"].replace({"GKP":"GK" })

In [129]:
# MINUTES PREDICTOR STARTS HERE************************************************
# Ensure both DataFrames have the same columns before alignment
# Create the date-related columns in 'test' first
import pandas as pd
from sklearn.model_selection import train_test_split

test["index"] = test["name"] + test["kickoff_time"].astype("str")
test = test.set_index("index")
test["date"] = pd.to_datetime(test["kickoff_time"], format='ISO8601', errors='coerce')
test["day_of week"] = test["date"].dt.day_name()  # Added parentheses to call the function
test["month"] = test["date"].dt.month
test["hour"] = test["date"].dt.hour
test["week"] = test["date"].dt.isocalendar().week
test.drop(["kickoff_time", "date"], axis=1, inplace=True)

# Now process 'train' DataFrame
# Create 'index' column *before* dropping 'kickoff_time'
train["index"] = train["name"] + train["kickoff_time"].astype("str")
train.drop_duplicates("index", keep="last", inplace=True)
train = train.set_index("index")

# Now you can create date features and then drop 'kickoff_time'
train["date"] = pd.to_datetime(train["kickoff_time"], format='ISO8601', errors='coerce')
train["day_of week"] = train["date"].dt.day_name()  # Added parentheses to call the function
train["month"] = train["date"].dt.month
train["hour"] = train["date"].dt.hour
train["week"] = train["date"].dt.isocalendar().week
train.drop(["kickoff_time", "date"], axis=1, inplace=True) # Now safe to drop


#Keep only common columns in both dataframes
common_cols = list(set(train.columns) & set(test.columns))

train = train[common_cols]
test = test[common_cols]



train_copy = train.copy()
test_copy = test.copy()


train["minutes"] = train["minutes"].apply(convert_minutes)

target = train[["minutes", "GW", "position"]]
train.drop(["total_points", "minutes"], axis=1, inplace=True)
test.drop(["total_points", "minutes"], axis=1, inplace=True)
train.drop(dropped_columns, axis=1, inplace=True)
test.drop(dropped_columns, axis=1, inplace=True)


# Convert categorical columns to numerical
for col in train.columns:
    if train[col].dtype == "object":
        if col not in ["team", "name", "position"]:
            train[col] = pd.factorize(train[col])[0]
            test[col] = pd.factorize(test[col])[0]

train["was_home"] = train["was_home"].replace({True: 0, False: 1})
test["was_home"] = test["was_home"].replace({True: 0, False: 1})

train = train[statistics + leak_columns + date_cols]
test = test[statistics + leak_columns + date_cols]

x, val, y, y_val = train_test_split(
    train.drop(leak_columns, axis=1),
    target["minutes"],
    test_size=0.1,
    random_state=0,
)

In [130]:
params={'colsample_bylevel': 0.8070621518153563, 'learning_rate': 0.04765984972709895, 'max_depth': 7, 'reg_lambda': 5, 'scale_pos_weight': 2.5,'subsample': 0.6794390204583894}
model=CatBoostClassifier(**params,cat_features=["position"],random_state=0,early_stopping_rounds=500,use_best_model=True,verbose=500,n_estimators=10000)

In [131]:
model.fit(x, y,eval_set=[(val,y_val)])

0:	learn: 0.6525880	test: 0.6526917	best: 0.6526917 (0)	total: 127ms	remaining: 21m 9s
500:	learn: 0.2974733	test: 0.3179153	best: 0.3179153 (500)	total: 1m 16s	remaining: 24m 14s
1000:	learn: 0.2735103	test: 0.3177685	best: 0.3176982 (972)	total: 2m 34s	remaining: 23m 6s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.3176981932
bestIteration = 972

Shrink model to first 973 iterations.


In [132]:
print(confusion_matrix(model.predict(val), y_val))
print(
    f"starting Accuracy score {accuracy_score(model.predict(val), y_val)}"
)

print(
    f" starting f1 score: {f1_score(model.predict(val), y_val)}"
)

feature_importance = pd.DataFrame(
    {"column": x.columns, "imp": model.feature_importances_}
).sort_values(
    "imp", ascending=False
)  #

[[5495  375]
 [1401 3920]]
starting Accuracy score 0.8413010454829773
 starting f1 score: 0.8153078202995009


[[3863  267]
 [ 975 2862]]
starting Accuracy score 0.8441069411321702
 starting f1 score: 0.8217054263565893

In [ ]:
feature_importance.head(50)

In [ ]:
feature_importance.tail(20)

In [ ]:
y_val

In [135]:
test_copy["minutes"] = model.predict(test.drop(leak_columns, axis=1))
test_copy[leak_columns + ["minutes"]].to_csv(
    f"minutes.csv"
)

In [136]:
predicted_minutes=model.predict(val)
val_=pd.DataFrame({"ind":val.index,"actul_minutes":y_val,"predicted_minutes":predicted_minutes,"position":val["position"]})

In [137]:
print(f"STARTING GOALKEEPERS PERFORMANCE!!!")
a=val_[val_["position"]=="GK"]["actul_minutes"]
b=val_[val_["position"]=="GK"]["predicted_minutes"]
print(confusion_matrix(a,b))
print(
    f"starting Accuracy score {accuracy_score(a,b)}"
)

print(
    f" starting f1 score: {f1_score(a,b)}"
)

STARTING GOALKEEPERS PERFORMANCE!!!
[[793  44]
 [ 27 324]]
starting Accuracy score 0.9402356902356902
 starting f1 score: 0.9012517385257302


STARTING GOALKEEPERS PERFORMANCE!!!
[[583  21]
 [ 24 216]]
starting Accuracy score 0.9466824644549763
 starting f1 score: 0.9056603773584907

STARTING DEFENDERS PERFORMANCE!!!
[[1449  208]
 [ 200  915]]
starting Accuracy score 0.8528138528138528
 starting f1 score: 0.817694369973190

STARTING MIDFIELDERS PERFORMANCE!!!
[[1595  288]
 [ 235 1195]]
starting Accuracy score 0.8421370359191066
 starting f1 score: 0.820460006865774

STARTING FORWARDS PERFORMANCE!!!
[[497  88]
 [ 69 325]]
starting Accuracy score 0.839632277834525
 starting f1 score: 0.8054522924411399

In [138]:
print(f"STARTING DEFENDERS PERFORMANCE!!!")
a=val_[val_["position"]=="DEF"]["actul_minutes"]
b=val_[val_["position"]=="DEF"]["predicted_minutes"]
print(confusion_matrix(a,b))
print(
    f"starting Accuracy score {accuracy_score(a,b)}"
)

print(
    f" starting f1 score: {f1_score(a,b)}"
)

STARTING DEFENDERS PERFORMANCE!!!
[[1843  543]
 [ 146 1282]]
starting Accuracy score 0.819349764027268
 starting f1 score: 0.7881955118352291


In [139]:
print(f"STARTING MIDFIELDERS PERFORMANCE!!!")
a=val_[val_["position"]=="MID"]["actul_minutes"]
b=val_[val_["position"]=="MID"]["predicted_minutes"]
print(confusion_matrix(a,b))
print(
    f"starting Accuracy score {accuracy_score(a,b)}"
)

print(
    f" starting f1 score: {f1_score(a,b)}"
)

STARTING MIDFIELDERS PERFORMANCE!!!
[[2185  630]
 [ 148 1821]]
starting Accuracy score 0.8373745819397993
 starting f1 score: 0.8239819004524886


In [140]:
print(f"STARTING FORWARDS PERFORMANCE!!!")
a=val_[val_["position"]=="FWD"]["actul_minutes"]
b=val_[val_["position"]=="FWD"]["predicted_minutes"]
print(confusion_matrix(a,b))
print(
    f"starting Accuracy score {accuracy_score(a,b)}"
)

print(
    f" starting f1 score: {f1_score(a,b)}"
)

STARTING FORWARDS PERFORMANCE!!!
[[674 184]
 [ 54 493]]
starting Accuracy score 0.8306049822064057
 starting f1 score: 0.8055555555555556


In [141]:
test_copy[test_copy["minutes"]==1]["position"].value_counts()

,count
position,
MID,169
DEF,114
FWD,37
GKP,24


In [142]:
train = train_copy[train_copy["minutes"] > 0]
test = test_copy[test_copy["minutes"] > 0]

# MINUTES PREDICTION ENDS HERE - MAKE SURE TO SAVE MINUTES PREDICTIONS AT THE BOTTOM OF THE SCRIPT

In [111]:
#Predict Points - MAKE SURE TO READ THE DATA IN AGAIN

train["index"] = train["name"] + train["kickoff_time"].astype("str")
train.drop_duplicates("index", keep="last", inplace=True)
train= train.set_index("index")
#train["date"]=pd.to_datetime(train["kickoff_time"])
train["date"] = pd.to_datetime(train["kickoff_time"], format='ISO8601', errors='coerce')
#train["day_of week"]=train["date"].dt.day_name
train["day_of week"] = train["date"].dt.day_name()  # Added parentheses to call the function
train["month"]=train["date"].dt.month
train["hour"]=train["date"].dt.hour
#train["week"]=train["date"].dt.week
train["week"] = train["date"].dt.isocalendar().week
train.drop(["kickoff_time","date"], axis=1, inplace=True)

test["index"] = test["name"] + test["kickoff_time"].astype("str")
test = test.set_index("index")
#test["date"]=pd.to_datetime(test["kickoff_time"])
test["date"] = pd.to_datetime(test["kickoff_time"], format='ISO8601', errors='coerce')
#test["day_of week"]=test["date"].dt.day_name
test["day_of week"] = test["date"].dt.day_name()  # Added parentheses to call the function
test["month"]=test["date"].dt.month
test["hour"]=test["date"].dt.hour
#test["week"]=test["date"].dt.week
test["week"] = test["date"].dt.isocalendar().week
test.drop(["kickoff_time","date"], axis=1, inplace=True)

target = train[["total_points", "GW","position" ]]
train.drop(["total_points", "minutes"], axis=1, inplace=True)
test.drop(["total_points", "minutes"], axis=1, inplace=True)
train.drop(dropped_columns, axis=1, inplace=True)
test.drop(dropped_columns, axis=1, inplace=True)


for col in train.columns:
    if train[col].dtype == "object":
        if col not in ["team", "name","position"]:
            train[col] = pd.factorize(train[col])[0]
            test[col] = pd.factorize(test[col])[0]

train["position"]=train["position"].astype("category")
test["position"]=test["position"].astype("category")
train["was_home"] = train["was_home"].replace({True: 0, False: 1})
test["was_home"] = test["was_home"].replace({True: 0, False: 1})

train = train[statistics + leak_columns+date_cols]
test= test[statistics + leak_columns+date_cols]

x, val, y, y_val = train_test_split(
    train.drop(leak_columns, axis=1),
    target["total_points"],
    test_size=0.1,
    random_state=0,
)

In [112]:
from sklearn.model_selection import KFold

#cross_validator to splite the data into folds
folds=KFold(n_splits=8,shuffle=True,random_state=0)

#a dataframe to store the predictions made by each fold
predictions_df=pd.DataFrame()

#list to save the mean absolute errors from validatingon each folds
rmse_val=[]
rmse_X=[]

#a simple catboost regressor
model=LGBMRegressor(**{'colsample_bytree': 0.4199299182268318, 'learning_rate': 0.0032874466037521254, 'max_depth': 9, 'min_split_gain': 0.5685369160138952, 'num_leaves': 99, 'reg_alpha': 0.5621526419488447, 'reg_lambda': 0, 'subsample': 0.6534153111773866}, verbose=-50,random_state=0,early_stopping_rounds=200,n_estimators=10000)

#train model, make predictions and check the validation accuracy on  each fold
for i,(train_index,test_index) in enumerate(folds.split(train.drop(leak_columns, axis=1),target["total_points"])):
    train_fold=train.drop(leak_columns, axis=1).iloc[train_index]
    val_fold=train.drop(leak_columns, axis=1).iloc[test_index]
    y_fold=target["total_points"].iloc[train_index]
    y_val_fold=target["total_points"].iloc[test_index]


    model.fit(train_fold,y_fold,eval_set=[(val_fold,y_val_fold)])
    print(i+1)
    prediction=model.predict(test.drop(leak_columns, axis=1))
    predictions_df[i]=prediction
    rmse_val.append(mean_squared_error(model.predict(val_fold),y_val_fold,squared=False))
    rmse_X.append(mean_squared_error(model.predict(train_fold),y_fold,squared=False))
print(rmse_val)
print(rmse_X)

1
2
3
4
5
6
7
8
[1.9946393170356036, 1.9546758580015136, 1.9780176949173842, 1.951287106223024, 2.004787998844915, 1.97115242637412, 1.959004215500157, 1.9723968096208457]
[1.454531659417867, 1.5038361988405857, 1.5270988350311632, 1.3766886402265621, 1.399895350872374, 1.490484348292099, 1.529895375944153, 1.7247040730961758]


In [113]:
print(np.mean(rmse_val))
print(np.mean(rmse_X))
#2.7575293874474336
#2.0174134721607295

1.9732451783146954
1.5008918102151225


2.7320675301713573
1.6917027774360753

In [114]:
predictions_df[0]

,0
0,0.023224
1,0.989653
2,0.056020
3,0.043454
4,2.557844
...,...
665,1.596976
666,2.271958
667,0.348180
668,4.683585


In [115]:
test["points"]=np.mean(predictions_df, axis=1).values

test[leak_columns + ["points", "value"]].sort_values(
    "points", ascending=False
).to_csv("points.csv")

In [116]:
test[test["position"]=="MID"].sort_values(by="points",ascending=False).head(11)[["name","points","team"]]

,name,points,team
index,,,
Mohamed Salah2024-11-02T15:00:00Z,Mohamed Salah,7.046866,Liverpool
Bukayo Saka2024-11-02T12:30:00Z,Bukayo Saka,5.566477,Arsenal
Bryan Mbeumo2024-11-04T20:00:00Z,Bryan Mbeumo,4.812529,Brentford
Cole Palmer2024-11-03T16:30:00Z,Cole Palmer,4.477835,Chelsea
Declan Rice2024-11-02T12:30:00Z,Declan Rice,4.459019,Arsenal
Brennan Johnson2024-11-03T14:00:00Z,Brennan Johnson,4.187138,Spurs
Eberechi Eze2024-11-02T17:30:00Z,Eberechi Eze,4.182510,Crystal Palace
Jarrod Bowen2024-11-02T15:00:00Z,Jarrod Bowen,4.021720,West Ham
Mikkel Damsgaard2024-11-04T20:00:00Z,Mikkel Damsgaard,3.893360,Brentford


In [117]:
test[test["position"]=="DEF"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
Trent Alexander-Arnold2024-11-02T15:00:00Z,Trent Alexander-Arnold,3.686967,Liverpool
Rayan Aït-Nouri2024-11-02T17:30:00Z,Rayan Aït-Nouri,3.640290,Wolves
Joško Gvardiol2024-11-02T15:00:00Z,Joško Gvardiol,3.541818,Man City
Ashley Young2024-11-02T15:00:00Z,Ashley Young,3.497327,Everton
Ethan Pinnock2024-11-04T20:00:00Z,Ethan Pinnock,3.232022,Brentford
Leif Davis2024-11-02T15:00:00Z,Leif Davis,3.206921,Ipswich
Murillo Santiago Costa dos Santos2024-11-02T15:00:00Z,Murillo Santiago Costa dos Santos,3.121346,Nott'm Forest
Ola Aina2024-11-02T15:00:00Z,Ola Aina,3.111873,Nott'm Forest
Nélson Cabral Semedo2024-11-02T17:30:00Z,Nélson Cabral Semedo,3.062821,Wolves


In [118]:
test[test["position"]=="GKP"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
Mark Flekken2024-11-04T20:00:00Z,Mark Flekken,3.757804,Brentford
Dean Henderson2024-11-02T17:30:00Z,Dean Henderson,3.516444,Crystal Palace
Jordan Pickford2024-11-02T15:00:00Z,Jordan Pickford,3.392530,Everton
Matz Sels2024-11-02T15:00:00Z,Matz Sels,3.361403,Nott'm Forest
Arijanet Muric2024-11-02T15:00:00Z,Arijanet Muric,3.243361,Ipswich
Ederson Santana de Moraes2024-11-02T15:00:00Z,Ederson Santana de Moraes,3.139257,Man City
André Onana2024-11-03T16:30:00Z,André Onana,3.016022,Man Utd
Guglielmo Vicario2024-11-03T14:00:00Z,Guglielmo Vicario,3.008410,Spurs
Bernd Leno2024-11-04T20:00:00Z,Bernd Leno,2.966050,Fulham


In [119]:
test[test["position"]=="FWD"].sort_values(by="points",ascending=False).head(10)[["name","points","team"]]

,name,points,team
index,,,
Ollie Watkins2024-11-03T14:00:00Z,Ollie Watkins,5.960358,Aston Villa
Erling Haaland2024-11-02T15:00:00Z,Erling Haaland,5.904990,Man City
Chris Wood2024-11-02T15:00:00Z,Chris Wood,5.085697,Nott'm Forest
Kai Havertz2024-11-02T12:30:00Z,Kai Havertz,4.155948,Arsenal
Matheus Santos Carneiro Da Cunha2024-11-02T17:30:00Z,Matheus Santos Carneiro Da Cunha,4.040393,Wolves
Nicolas Jackson2024-11-03T16:30:00Z,Nicolas Jackson,4.014294,Chelsea
Jørgen Strand Larsen2024-11-02T17:30:00Z,Jørgen Strand Larsen,3.692594,Wolves
Raúl Jiménez2024-11-04T20:00:00Z,Raúl Jiménez,3.692124,Fulham
Jamie Vardy2024-11-02T15:00:00Z,Jamie Vardy,3.279927,Leicester


In [120]:
test["points"].sort_values(ascending=False).head(50)

,points
index,
Mohamed Salah2024-11-02T15:00:00Z,7.046866
Ollie Watkins2024-11-03T14:00:00Z,5.960358
Erling Haaland2024-11-02T15:00:00Z,5.904990
Bukayo Saka2024-11-02T12:30:00Z,5.566477
Chris Wood2024-11-02T15:00:00Z,5.085697
Bryan Mbeumo2024-11-04T20:00:00Z,4.812529
Cole Palmer2024-11-03T16:30:00Z,4.477835
Declan Rice2024-11-02T12:30:00Z,4.459019
Brennan Johnson2024-11-03T14:00:00Z,4.187138


In [121]:

feature_importance = pd.DataFrame(
    {"column": x.columns, "imp": model.feature_importances_}
).sort_values(
    "imp", ascending=False
)  #

In [122]:
feature_importance.head(50)

,column,imp
71,opp mean team Goal scored all,6558
72,opp mean team Goal conceded all,6255
73,opp mean match_result all,6133
29,mean ict_index 3,5027
4,percent_value,4832
64,mean team Goal scored all,4792
66,mean team Goal conceded all,4760
26,mean creativity 3,4661
24,mean bps 3,4639
62,mean match_result all,4487


In [123]:
feature_importance.tail(30)

,column,imp
9,minutes_ex,2739
12,influence_ex,2708
63,mean team Goal scored 3,2639
10,goals_conceded_ex,2622
76,hour,2435
3,last_season_position,2428
27,mean goals_conceded 3,2133
14,bonus_ex,1948
65,mean team Goal conceded 3,1850
17,clean_sheets_ex,1805


#Save predictions

In [143]:
test_copy[test_copy["position"]=="DEF"][["name","team","minutes"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/defendetrs_minutes.csv")
test_copy[test_copy["position"]=="GKP"][["name","team","minutes"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/goalkeepers_minutes.csv")
test_copy[test_copy["position"]=="MID"][["name","team","minutes"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/midfielders_minutes.csv")
test_copy[test_copy["position"]=="FWD"][["name","team","minutes"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/forwards_minutes.csv")

In [124]:
test[test["position"]=="DEF"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/defenders_points.csv")
test[test["position"]=="GKP"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/goalkeepers_points.csv")
test[test["position"]=="MID"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/midfielders_points.csv")
test[test["position"]=="FWD"][["name","team","points","value"]].to_csv(f"/content/gdrive/MyDrive/fplpredict/predicted_dataset/GW{gameweek}/forwards_points.csv")